In [ ]:
import os
import numpy as np
import pickle as pkl
from openai import OpenAI

## Load Dataset

In [ ]:
city = 'ny'

city_full_name = {
    'ny': 'New York City',
    'hs': 'Houston',
    'sf': 'San Francisco'
}

In [ ]:
with open('indices.pkl', 'rb') as f:
    indices = pkl.load(f)
    
with open('dates.pkl', 'rb') as f:
    dates = pkl.load(f)
    
with open(f'time_series_{city}.pkl', 'rb') as f:
    data = pkl.load(f)

In [ ]:
data_size = data.shape[0]
window_size = 24
print(data_size, window_size, len(indices))

## Prompt GPT4

In [ ]:
API_KEY = ''

In [ ]:
system_prompt = f"Your job is to act as a professional weather analyst. You will write a high-quality report that is informative and helps in understanding the current weather situation."
print(system_prompt)

In [ ]:
client = OpenAI(api_key=API_KEY)

In [ ]:
for i in indices:
    
    data_window = data[i:i+window_size]
    
    humidity = '|'.join([f"{x:.2f}" for x in data_window[:,0]])
    pressure = '|'.join([f"{x:.2f}" for x in data_window[:,1]])
    temperature = '|'.join([f"{x:.2f}" for x in data_window[:,2]])
    wind_speed = '|'.join([f"{x:.2f}" for x in data_window[:,3]])
    wind_direction = '|'.join([f"{x:.2f}" for x in data_window[:,4]])

    user_prompt = f"Your task is to analyze key weather indicators in {city_full_name[city]} over the last {window_size} hours."
    user_prompt += f"\n\nReview the time-series data provided for the last {window_size} hours. "
    user_prompt += f"Each time-series consists of hourly values separated by a \'|\' token for the following indicators:\n"
    user_prompt += f"- Temperature (Kelvin): {temperature}\n- Humidity (%): {humidity}\n- Air Pressure (hPa): {pressure}\n- Wind Speed (m/s): {wind_speed}\n- Wind Direction (degrees): {wind_direction}\n\n"
    user_prompt += f"Based on this time-series data, write a concise report that provides insights crucial for understanding the current weather situation. "
    user_prompt += f"Your report should be limited to five sentences, yet comprehensive, highlighting key trends and considering their potential impact on the weather in {city_full_name[city]}."
    user_prompt += f"Do not write numerical values while writing the report."
    
    response = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=[
        {
          "role": "system",
          "content": system_prompt
        },
        {
          "role": "user",
          "content": user_prompt
        }
        ],
        temperature=0.7,
        max_tokens=2048,
        top_p=1
    )

    text = response.choices[0].message.content

    with open(f'gpt_summary/{city}_{i}.txt', 'w') as f:
        f.write(text)